Otimização da Função de Ackley

Grupo: Anderson Carneiro da Silva (acs6@cin.ufpe.br) e Lucas Thierry Chaves Muniz (ltcm@cin.ufpe.br)

In [16]:
from numpy import pi
from numpy import cos
from numpy import exp
from numpy import sqrt

Função Fitness

In [17]:
def ackley (vector, c1=20, c2=0.2, c3=2*pi):
    d = len(vector)
    sqrt_sum=0
    cos_sum=0
    for elem in vector:
        sqrt_sum += elem**2
        cos_sum += cos(c3*elem)
    
    sqrt_term = -c1 * exp(-c2*(sqrt(sqrt_sum/d)))
    cos_term = -exp(cos_sum/d)

    return sqrt_term + cos_term + c1 + exp(1)

In [18]:
print(ackley([0, 0, 0, 0]))

4.440892098500626e-16


In [19]:
import numpy as np
from abc import ABC, abstractmethod


class Individual(ABC):
    def __init__(self, value=None, init_params=None):
        if value is not None:
            self.value = value
        else:
            self.value = self._random_init(init_params)

    @abstractmethod
    def pair(self, other, pair_params):
        pass

    @abstractmethod
    def mutate(self, mutate_params):
        pass

    @abstractmethod
    def _random_init(self, init_params):
        pass


class Optimization(Individual):
    def pair(self, other, pair_params):
        return Optimization(pair_params['alpha'] * self.value + (1 - pair_params['alpha']) * other.value)

    def mutate(self, mutate_params):
        self.value += np.random.normal(0, mutate_params['rate'], mutate_params['dim'])
        for i in range(len(self.value)):
            if self.value[i] < mutate_params['lower_bound']:
                self.value[i] = mutate_params['lower_bound']
            elif self.value[i] > mutate_params['upper_bound']:
                self.value[i] = mutate_params['upper_bound']

    def _random_init(self, init_params):
        return np.random.uniform(init_params['lower_bound'], init_params['upper_bound'], init_params['dim'])


class Population:
    def __init__(self, size, fitness, individual_class, init_params):
        self.fitness = fitness
        self.individuals = [individual_class(init_params=init_params) for _ in range(size)]
        self.individuals.sort(key=lambda x: self.fitness(x))

    def replace(self, new_individuals):
        size = len(self.individuals)
        self.individuals.extend(new_individuals)
        self.individuals.sort(key=lambda x: self.fitness(x))
        self.individuals = self.individuals[-size:]

    def get_parents(self, n_offsprings):
        mothers = self.individuals[-2 * n_offsprings::2]
        fathers = self.individuals[-2 * n_offsprings + 1::2]

        return mothers, fathers


class Evolution:
    def __init__(self, pool_size, fitness, individual_class, n_offsprings, survival_select, pair_params, mutate_params, init_params):
        self.pair_params = pair_params
        self.mutate_params = mutate_params
        self.pool = Population(pool_size, fitness, individual_class, init_params)
        self.n_offsprings = n_offsprings
        self.survival_select=survival_select

    def step(self):
        mothers, fathers = self.pool.get_parents(self.n_offsprings)
        offsprings = []

        for mother, father in zip(mothers, fathers):
            offspring = mother.pair(father, self.pair_params)
            offspring.mutate(self.mutate_params)
            offsprings.append(offspring)
            if(self.survival_select=='plus'):
                offsprings.append(Optimization(mother.value))
                offsprings.append(Optimization(father.value))

        if(self.survival_select=='comma'):
            self.pool.replace(offsprings)
        elif(self.survival_select=='plus'):
            self.pool.replace(offsprings)

In [20]:
def fitness(vector):
    return 1/ackley(vector.value)


xi_min = -15
xi_max = 15
dimensions = 30

evo = Evolution(
    pool_size=30, fitness=fitness, individual_class=Optimization, n_offsprings=15, survival_select='plus',
    pair_params={'alpha': 0.5},
    mutate_params={'lower_bound': xi_min, 'upper_bound': xi_max, 'rate': 0.5, 'dim': dimensions},
    init_params={'lower_bound': xi_min, 'upper_bound': xi_max, 'dim': dimensions}
)
generations = 10000

for i in range(generations):
    evo.step()

best_individual = evo.pool.individuals[-1]
print(best_individual.value)
print(fitness(best_individual))

[ 0.2351786   1.16869777 -0.23889359 -0.04277537  0.08296662 -0.10120102
  0.01348354 -0.05431364  0.79860028 -0.22399718  0.06126839  0.22830242
 -0.05785574 -0.8978244   0.7796644  -0.12095286 -0.04052891  0.95244219
 -0.14833107  0.15169603 -0.04098997  0.08046448 -0.04992963 -0.44914736
  0.29697698  0.03123265 -0.10524447  0.79864062 -0.57890327  0.07115664]
0.3695668586798421
